# A 256 Bit PseudoRandom Number.

In order to generate a **random number** we should initialize the random number generator. It's like a set full of random numbers that are drawn one after the another.

In [1]:
open System

let rnd = Random()

printfn "%A" (rnd.Next ())
printfn "%A" (rnd.Next ())


2033664313
944397660


## Seeds
We can feed the random number generator with a seed, an **integer** that changes the set of random numbers to draw from.

In [2]:
let seed = 12
rnd = Random(seed) //It's mutable!!!!

printfn "%A" (rnd.Next ())
printfn "%A" (rnd.Next ())
printfn "%A" (rnd.NextDouble ())
printfn "%A" (rnd.NextDouble ())



1589768563
1741816558
0.09466343284
0.3703295255


## Limits
We can limit the number drawn from the set giving a range. Like a random integer beetween 0 and 12. In this case we want to generate a 256 bit number bit by bit, so we're going to limit the range beetwen 0 and 1.

In [3]:
let pseudo_number_256 = [0..255] |> List.map (fun x -> rnd.Next (0,2))

printfn "%A" pseudo_number_256

[1; 0; 1; 1; 0; 0; 1; 1; 0; 0; 1; 1; 1; 0; 1; 0; 1; 0; 1; 1; 0; 0; 1; 1; 0; 0; 0;
 1; 0; 0; 1; 1; 0; 1; 0; 1; 1; 1; 0; 1; 0; 1; 0; 1; 1; 0; 1; 1; 1; 0; 1; 1; 0; 0;
 1; 0; 0; 0; 0; 1; 1; 1; 0; 1; 0; 1; 0; 1; 1; 0; 1; 1; 1; 1; 1; 1; 1; 0; 1; 0; 1;
 1; 0; 1; 1; 0; 1; 0; 1; 1; 1; 0; 0; 1; 0; 0; 1; 0; 0; 0; ...]


And we can glue together all the elements of this list in a unique number.

In [4]:
let number = pseudo_number_256 |> List.map (sprintf "%i") |> String.concat ""
printfn "%A" number

"0000100000100111101100011111110000111011110010001110110011111111111100111110100001000100101011000101011010101000001001010111001011000111110000010001111000111101010110010111001100101100011100011010011111000100011101111010001011001011010000111100001100110111"


0.08441184532
0.5005662261


## Problem
In case you need a random value to be used in cryptography such as a cryptographic key in symmetric and asymmetric encryption then [System.Random](https://msdn.microsoft.com/it-it/library/system.random.aspx) is not an acceptable option. Infact it represents a pseudo-random number generator, which is a device that produces a sequence of numbers that meet certain statistical requirements for randomness. [Here is the code.](https://referencesource.microsoft.com/#mscorlib/system/random.cs,bb77e610694e64ca)
We're going to use instead the RNGCryptoServiceProvider Class.


In [6]:
open System.Security.Cryptography

//Generate a Random Cryptographic Key
let getKey (keyLength:int) = 
    let rng = new RNGCryptoServiceProvider()
    let mutable randomBytes =  Array.create keyLength 0uy   //mutable, otherwise <null> !!!!
    rng.GetBytes randomBytes  // Fill the array with a random value.
    randomBytes
    
let skey (len:int)=  Convert.ToBase64String(getKey(len));
(*
printfn "%A" (getKey 32) //256 bit number!!!
printfn "%A" (getKey 256)
*)

printfn "32 bytes %A" (skey 32)
printfn "256 bytes %A" (skey 256)


32 bytes "82cVZEQ2cS1gBzv3kEqofhAp2+9Atlf0Lf6ykge83rU="
256 bytes "xIZMheUqpRD5mq9mPoAlg4RQDu7178y7QsY3/39qitaFKQpN92xpfcnN2rMGU+vf1ji//Tz6NtefRejvXmDh3q60iUOUKbRaPd9imlrE+cOHtpio1iqYhAFVpSaFChSY5fXMD4DOVDJCFf/rVt+pv4xKQsckIwgYXhjAwVEL1W0UiydrxwwBlXkfTdJqvnK/s00HhXdnHR5CxbIcEZ5lFJ48xAoA6aPXzUytNfjun1OTczd5JeA61GOt2VjJJtK1hINCZ4HjoRxgXhUDH8xlpoMawtTFyMx3TD0mnbbDfjMTVeXhtWs5QEjO2KIUSzDT73Au13opCqd3JHgF9XILpw=="


## Base58

The Base58 encoding algorithm is similar to Base64, but we remove certain characters that cause issues with URLs, and **cause confusion** because of how similar they look in certain fonts. [Base58Check](https://github.com/adamcaudill/Base58Check) adds a 4 byte checksum to validate that the data hasn't been altered in transmission. This checksum isn't suitable to perform cryptographic validation, but is does detect accidental corruption.

In [7]:
#load "Paket.fsx"
Paket.Package
  [ "Base58Check -Version 0.2.0" ]  
#load "Paket.Generated.Refs.fsx"

open Base58Check 

let private_key = getKey 32

printfn "%A" (private_key)

let b32 =  Base58CheckEncoding.Encode private_key //Encodes data with a 4-byte checksum
let pb32 = Base58CheckEncoding.EncodePlain private_key //Encodes data in plain Base58, without any checksum

printfn "%A" (b32)
printfn "%A" (pb32)

 

[|171uy; 2uy; 58uy; 160uy; 197uy; 175uy; 161uy; 4uy; 191uy; 216uy; 43uy; 121uy;
  134uy; 76uy; 214uy; 28uy; 193uy; 39uy; 95uy; 12uy; 227uy; 158uy; 28uy; 229uy;
  82uy; 19uy; 14uy; 129uy; 126uy; 213uy; 250uy; 134uy|]
"2JKBu7U19Ljcioee9sxgS4m9zbV3P191vroSV73dqEGrF5hfUv"
"CWYgtKfE1s3GEH5TWe6BJGj3PaQBejtfwsdmWm8WkpoP"


The original *Bitcoin client source* code explains the reasoning behind [base58 encoding](https://en.bitcoin.it/wiki/Base58Check_encoding):

base58.h:

        // Why base-58 instead of standard base-64 encoding?
        // - Don't want 0OIl characters that look the same in some fonts and
        //      could be used to create visually identical looking account numbers.
        // - A string with non-alphanumeric characters is not as easily accepted as an account number.
        // - E-mail usually won't line-break if there's no punctuation to break at.
        // - Doubleclicking selects the whole number as one word if it's all alphanumeric.

Encoding is not cryptography and is reversible. It's all about usability:

In [8]:
printfn "%A" (Base58CheckEncoding.Decode b32)
printfn "%A" (Base58CheckEncoding.DecodePlain pb32)


[|171uy; 2uy; 58uy; 160uy; 197uy; 175uy; 161uy; 4uy; 191uy; 216uy; 43uy; 121uy;
  134uy; 76uy; 214uy; 28uy; 193uy; 39uy; 95uy; 12uy; 227uy; 158uy; 28uy; 229uy;
  82uy; 19uy; 14uy; 129uy; 126uy; 213uy; 250uy; 134uy|]
[|171uy; 2uy; 58uy; 160uy; 197uy; 175uy; 161uy; 4uy; 191uy; 216uy; 43uy; 121uy;
  134uy; 76uy; 214uy; 28uy; 193uy; 39uy; 95uy; 12uy; 227uy; 158uy; 28uy; 229uy;
  82uy; 19uy; 14uy; 129uy; 126uy; 213uy; 250uy; 134uy|]


In [ ]:
#load "XPlot.Plotly.Paket.fsx"
#load "XPlot.Plotly.fsx"
open XPlot.Plotly

let rnds =  {0..10000} |> Seq.map (fun x-> rnd.NextDouble())

rnds
    |> Chart.Bar
    |> Chart.WithLayout (Layout(title = "Random numbers"))
    |> Chart.WithHeight 300
    |> Chart.WithWidth 400
    
    

In [ ]:
type auxValues = 
    {M2: float; mean : float; n : float
 
    }
    with
        member private a.Display = "M2: " + a.M2.ToString() + "; mean: " + a.mean.ToString() + "; n: " + a.n.ToString()
 
let auxValuesOfSet (data : float []) start finish =
    let mutable n = 0.
    let mutable mean = 0.
    let mutable M2 = 0.
 
    for x = start to finish do
            n <- n + 1.
            let delta = data.[x] - mean
            mean <- mean + delta/n
            if n > 1. then M2 <- M2 + delta * (data.[x] - mean) 
 
    {M2 = M2; mean = mean; n = float(finish - start + 1)}
 
let Variance (data : float []) =
    (auxValuesOfSet data 0 (data.Length - 1)).M2 / (float data.Length)
    
    
printfn "%A" (Variance (rnds |> Seq.toArray))

let avg = Seq.averageBy float rnds
printfn "%A" (avg)

Provare a generare i primi 100.000 numeri....

[1](https://blockchain.info/address/1EHNa6Q4Jz2uvNExL497mE43ikXhwF6kZm)

In [13]:
let numberToBytes(n:int) = 
      let intBytes:byte[] = BitConverter.GetBytes(n)
      if (BitConverter.IsLittleEndian) then
            Array.Reverse(intBytes)
      intBytes

let one = numberToBytes 1

printfn "%A" one

[|0uy; 0uy; 0uy; 1uy|]


[implementazione](https://github.com/MetacoSA/NBitcoin/blob/master/NBitcoin/Key.cs)